In [1]:
import tensorflow as tf
import numpy as np
import pickle as pkl
import sklearn as sk
from sklearn import preprocessing as pre
from pathos.pools import ProcessPool as Pool


/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#some hypers
GO='<START>'
STOP='<END>'
pad='<PAD>'
unknown='<UNKNOWN>'
BATCH=64
BEAM_WIDTH=5
EPOCHS=10

In [3]:
#load data
with open('./captions/token_train.pkl') as f:
    trainCaptions=pkl.load(f)
with open('./captions/token_dev.pkl') as f:
    devCaptions=pkl.load(f)
with open('./captions/token_test.pkl') as f:
    testCaptions=pkl.load(f)

In [4]:
def processToken(caps):
    nk=[]
    
    for i in caps:
        t=[GO]+list(i[1])+[STOP]
        empty=33-len(t)
        
        #t=t+[pad]*empty
        nk.append([i[0],t])
    return nk

In [5]:
trainCaptions=processToken(trainCaptions)
devCaptions=processToken(devCaptions)
testCaptions=processToken(testCaptions)

In [6]:
def listofwords(data):
    '''takes a list of sentences nd returns vocab'''
    a=[]
    for i in data:
        for j in i[1]:
            if j not in a:
                a.append(j)
    return a

In [7]:
#find the vocab and size
trainVocab=listofwords(trainCaptions+devCaptions)
trainVocabSize=len(trainVocab)

In [8]:
#find cap len
trainCapLen=[len(i[1]) for i in trainCaptions]
devCapLen=[len(i[1]) for i in devCaptions]
testCapLen=[len(i[1]) for i in testCaptions]

In [9]:
#Label Encoder for output transform
pre_op=pre.LabelEncoder()
pre_op.fit(trainVocab)
onehoter=np.identity(len(pre_op.classes_))

#word to int
trainSeq=[pre_op.transform(i[1]) for i in trainCaptions]
devSeq=[pre_op.transform(i[1]) for i in devCaptions]
trainID=[i[0] for i in trainCaptions]
devID=[i[0] for i in devCaptions]



In [10]:
#word to int with pool
p=Pool(8)


trainSeq=p.map(pre_op.transform,[i[1] for i in trainCaptions])
devSeq=p.map(pre_op.transform,[i[1] for i in devCaptions])
trainID=[i[0] for i in trainCaptions]
devID=[i[0] for i in devCaptions]



Process PoolWorker-4:
Process PoolWorker-8:
Process PoolWorker-3:
Process PoolWorker-6:
Process PoolWorker-7:
Traceback (most recent call last):
Process PoolWorker-1:
Process PoolWorker-5:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/multiprocess/process.py", line 267, in _bootstrap
Traceback (most recent call last):
Traceback (most recent call last):
Process PoolWorker-2:
    self.run()
  File "/usr/local/lib/python2.7/dist-packages/multiprocess/process.py", line 267, in _bootstrap
  File "/usr/local/lib/python2.7/dist-packages/multiprocess/process.py", line 267, in _bootstrap
Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/multiprocess/process.py", line 114, in run
Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/multiprocess/process.py", line 267, in _bootstrap
  File "/usr/local/lib/python2.7/dist-packages/multiprocess/process.py", line 267, in 

In [11]:
#appending stops

MAX_LEN=33
trainSeqReg=[np.pad(i,(0,MAX_LEN-len(i)),'constant',constant_values=pre_op.transform([STOP])) for i in trainSeq]
devSeqReg=[np.pad(i,(0,MAX_LEN-len(i)),'constant',constant_values=pre_op.transform([STOP])) for i in devSeq]
trainSeqReg=np.array(trainSeqReg)
devSeqReg=np.array(devSeqReg)

In [12]:
#loading video features
videoFeats=np.load(file='./features/consilidated_feats.npy')
videoFeatSize=np.array([len(i) for i in videoFeats])
#making the shape regular
videoFeats=np.array([np.pad(i,mode='constant',pad_width=[(0,28-len(i)),(0,0)]) for i in videoFeats])

In [13]:
#Lets build the graph
embedding_size=512
lstm_units=512
dropout_keep_prob=0.5
tf.reset_default_graph()

In [14]:
source_seq = tf.placeholder(shape=(None,28,2048),dtype=tf.float32)
target_seq = tf.placeholder(shape=(None,33),dtype=tf.int32)
source_seq_len = tf.placeholder(shape=(None,), dtype=tf.int32)
target_seq_len = tf.placeholder(shape=(None,), dtype=tf.int32)
no_start_target_seq = tf.placeholder(shape=(None,32),dtype=tf.int32)
batch_size = tf.placeholder(shape=(None),dtype=tf.int32)
real_target_seq_len= tf.placeholder(shape=(None,), dtype=tf.int32)
keep_prob= tf.placeholder(dtype=tf.float32)

In [15]:
#output embeddings
embedding_matrix_decode = tf.get_variable(
    name="embedding_matrix_de",
    shape=[trainVocabSize, embedding_size],
    dtype=tf.float32)
decoder_input_embedded = tf.nn.embedding_lookup(embedding_matrix_decode, target_seq) 


In [16]:
#output embeddings
embedding_matrix_decode = tf.Variable(initial_value=tf.random_normal(shape=[trainVocabSize, embedding_size],dtype=tf.float32))
decoder_input_embedded = tf.nn.embedding_lookup(embedding_matrix_decode, target_seq) 


In [17]:
#encoder
encoderCell=tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicLSTMCell(lstm_units),input_keep_prob=keep_prob,
                                          output_keep_prob=keep_prob)
encoder_outputs,encoder_final_state=tf.nn.dynamic_rnn(cell=encoderCell,inputs=source_seq,sequence_length=source_seq_len,
                 dtype=tf.float32)

#expri
#encoder_outputs_tiled=tf.contrib.seq2seq.tile_batch(encoder_outputs,multiplier=BEAM_WIDTH)
#encoder ends here

Instructions for updating:
Use the retry module or similar alternatives.


#exp
tiled_encoder_outputs = tf.contrib.seq2seq.tile_batch(
    encoder_outputs, multiplier=BEAM_WIDTH)
tiled_encoder_final_state = tf.contrib.seq2seq.tile_batch(
    encoder_final_state, multiplier=BEAM_WIDTH)
tiled_sequence_length = tf.contrib.seq2seq.tile_batch(
    source_seq_len, multiplier=BEAM_WIDTH)

#exp
attention_mechanism = tf.contrib.seq2seq.LuongAttention(lstm_units,encoder_outputs,memory_sequence_length=tiled_sequence_length)
attention_cell = tf.contrib.seq2seq.AttentionWrapper(tf.contrib.rnn.LSTMCell(lstm_units), attention_mechanism,attention_layer_size=lstm_units)
decoder_initial_state = attention_cell.zero_state(
    dtype=tf.float32, batch_size=BATCH * BEAM_WIDTH)
decoder_initial_state = decoder_initial_state.clone(
    cell_state=tiled_encoder_final_state)

In [18]:
#attention
attention_mechanism = tf.contrib.seq2seq.LuongAttention(lstm_units,encoder_outputs)

In [19]:
#Projection layer and decoder cell
output_layer = tf.layers.Dense(trainVocabSize)

decoder_cell=tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicLSTMCell(lstm_units),input_keep_prob=keep_prob,
                                          output_keep_prob=keep_prob)


In [20]:
decoder_initial_state=encoder_final_state

In [21]:
#decoder Attention wrapper
decoder_cell = tf.contrib.seq2seq.AttentionWrapper(
        decoder_cell, attention_mechanism,
        attention_layer_size=lstm_units,alignment_history=False)
decoder_initial_state = decoder_cell.zero_state(BATCH, tf.float32).clone(cell_state=decoder_initial_state)

In [22]:
#Training helper and decoder
helper = tf.contrib.seq2seq.TrainingHelper(decoder_input_embedded,target_seq_len)
decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, helper, initial_state=decoder_initial_state,output_layer=output_layer)#,output_layer=projection_layer)
outputs, state, seq_len = tf.contrib.seq2seq.dynamic_decode(decoder)
logits = outputs.rnn_output


In [23]:
#Inference helper(greedy) and decoder
helper2 = tf.contrib.seq2seq.GreedyEmbeddingHelper(embedding_matrix_decode,tf.fill([batch_size],
                                                    np.int32(pre_op.transform([GO])[0])),
                                                   np.int32(pre_op.transform([STOP])[0]))


decoder2 = tf.contrib.seq2seq.BasicDecoder(decoder_cell, helper2, decoder_initial_state,output_layer=output_layer)#,output_layer=projection_layer)

outputs, state, seq_len = tf.contrib.seq2seq.dynamic_decode(decoder2,maximum_iterations=32+10)

translations_logits = outputs.rnn_output
trs=outputs.sample_id

#Beam Search decoder
decoder_initial_state_tiled = tf.contrib.seq2seq.tile_batch(
    decoder_initial_state[0], multiplier=BEAM_WIDTH)

decoder_initial_state_tiled=decoder_cell.zero_state(batch_size=BATCH*BEAM_WIDTH,dtype=tf.float32).clone(cell_state=decoder_initial_state_tiled)


# Define a beam-search decoder
decoder3 = tf.contrib.seq2seq.BeamSearchDecoder(
        cell=decoder_cell,
        embedding=embedding_matrix_decode,
        start_tokens=tf.fill([batch_size],np.int32(pre_op.transform([GO])[0])),
        end_token=np.int32(pre_op.transform([STOP])[0]),
        initial_state=decoder_initial_state_tiled,
        beam_width=BEAM_WIDTH,
        output_layer=output_layer,
        length_penalty_weight=0.0)
outputs, state, seq_len = tf.contrib.seq2seq.dynamic_decode(decoder3,maximum_iterations=32+10)


trs_beam=outputs.predicted_ids


In [24]:
#loss and optimizer
cross_entropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=no_start_target_seq,logits=logits)

target_weights = tf.sequence_mask(real_target_seq_len, target_seq_len[0], dtype=logits.dtype)

loss=tf.reduce_sum(cross_entropy*target_weights)
train = tf.train.AdamOptimizer().minimize(loss)


In [25]:
#dont touch
maxtlen=max(trainCapLen)
maxvlen=max(devCapLen)
t_newlen=[maxtlen-1 for i in range(len(trainCapLen))]
v_newlen=[maxtlen-1 for i in range(len(devCapLen))]

In [26]:
sess=tf.InteractiveSession()
tf.global_variables_initializer().run()

In [27]:
#makes training batch
def getTrainBatch(indexs):
    sourceBatch=np.array([videoFeats[trainID[i]] for i in indexs])
    targetBatch=np.array([trainSeqReg[i] for i in indexs])
    sourceBatchLen=np.array([videoFeatSize[trainID[i]] for i in indexs])
    targetBatchLen=np.array([trainCapLen[i] for i in indexs])
    return sourceBatch,targetBatch,sourceBatchLen,targetBatchLen

#makes dev batch
def getDevBatch(indexs):
    sourceBatch=np.array([videoFeats[devID[i]] for i in indexs])
    targetBatch=np.array([devSeqReg[i] for i in indexs])
    sourceBatchLen=np.array([videoFeatSize[devID[i]] for i in indexs])
    targetBatchLen=np.array([devCapLen[i] for i in indexs])
    return sourceBatch,targetBatch,sourceBatchLen,targetBatchLen

In [37]:
#training starts here
training_losses=[]
valid_losses=[]
tData=np.arange(len(trainSeqReg))
dData=np.arange(len(devSeqReg))
for j in range(EPOCHS):
    np.random.shuffle(tData) #makes them iid
    training_loss=0
    for i in range(len(trainSeqReg)/BATCH):
        start=i*BATCH
        stop=(i+1)*BATCH
        sourceBatch,targetBatch,sourceBatchLen,targetBatchLen=getTrainBatch(tData[start:stop])
        _,lost=sess.run([train,loss],feed_dict={source_seq:sourceBatch,
                                                target_seq:targetBatch,
                                              source_seq_len:sourceBatchLen,
                                                target_seq_len:t_newlen[start:stop],
                                                real_target_seq_len:targetBatchLen,
                                                no_start_target_seq:np.array(targetBatch)[:,1:],
                                                batch_size:BATCH,keep_prob:dropout_keep_prob
                                                })
        
        training_loss+=lost
        print lost,
    #calculate t_loss
    training_losses.append(training_loss/len(trainSeqReg))
    
    #calculate v_loss
    validation_loss=0
    for k in range(len(devSeqReg)/BATCH):
        start=k*BATCH
        stop=(k+1)*BATCH
        sourceBatch,targetBatch,sourceBatchLen,targetBatchLen=getDevBatch(dData[start:stop])
        lost=sess.run(loss,feed_dict={source_seq:sourceBatch,
                                                target_seq:targetBatch,
                                              source_seq_len:sourceBatchLen,
                                                target_seq_len:t_newlen[start:stop],
                                                real_target_seq_len:targetBatchLen,
                                                no_start_target_seq:np.array(targetBatch)[:,1:],
                                                batch_size:BATCH,keep_prob:1.0
                                                })
        validation_loss += lost
        
    valid_losses.append(validation_loss/len(devSeqReg))
    
    print "Epoch:%d training loss%.4f: valid loss:%.4f"% (j,training_losses[-1],valid_losses[-1])
    #print "Epoch:%d training loss%.4f"% (j,training_losses[-1])

2039.0929 2031.2197 1992.6829 2200.6194 1830.4913 1923.2778 2113.1716 1724.2805 1751.8956 2019.3406 1975.0178 1829.1504 1855.862 1987.935 1925.7661 2257.9521 1817.3787 2160.111 1966.7537 1954.9543 2138.918 1626.8542 1873.8922 1906.4272 1852.7139 1880.9371 1812.5327 1876.3601 1972.252 1999.8743 1883.8611 1853.1763 1726.6484 2027.0605 2018.5614 2048.0967 1909.0339 2088.9863 2085.8179 1863.9617 1980.9109 1669.9487 1996.4661 1669.584 1852.824 2187.273 1745.2544 2047.8574 1771.6672 1793.3269 1710.2079 1785.215 1798.1599 2186.5317 1680.4185 1846.0629 2030.6459 1718.0083 1597.9392 2258.7954 2042.1947 1949.8701 1721.9219 1753.0767 1880.7051 1969.6182 2051.9321 2077.6155 1870.887 1934.396 1907.4114 1876.7751 1689.1366 1766.0061 1904.6366 1932.3247 2240.288 1885.5095 1736.5474 1775.9868 1883.3896 1874.6527 1953.4996 1977.0253 1821.3804 2122.8042 1979.1462 2283.1538 1740.8611 2125.3428 1919.2289 2148.9253 1890.6543 1823.7942 2088.886 1648.1584 2037.7435 2041.9622 2023.6122 2031.0522 1596.8386 222

KeyboardInterrupt: 

data=videoFeats[1300:1364]
data_len=videoFeatSize[1300:1364]
gen_sum=[]
for i in range(len(data)/BATCH):
    start=0
    stop=64
    
    load_trs=trs_beam
    y=sess.run(load_trs,feed_dict={source_seq:data,
                                               source_seq_len:data_len,
                                              batch_size:BATCH,keep_prob:1.0
                                                })
    print y
    
    
    #y=y[:,:,0]
    
    #for t in y:
     #   gen_sum.append(t)



In [38]:
data=videoFeats[1300:1364]
data_len=videoFeatSize[1300:1364]
gen_sum=[]
for i in range(len(data)/BATCH):
    start=i*BATCH
    stop=(i+1)*BATCH
    load_trs=trs
    y=sess.run(load_trs,feed_dict={source_seq:data[start:stop],
                                               source_seq_len:data_len[start:stop],
                                              batch_size:BATCH,
                                                keep_prob:1.0
                                                })
    for t in y:
        gen_sum.append(t)


In [39]:
#processing summaries
summs=[]
for i in gen_sum:
    summ=''
    for j in i:
        
        if j!=58:
            summ = summ+' '+pre_op.inverse_transform(j)
    summs.append(summ[1:])

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

In [40]:
summs

['a man is cutting a potato',
 'a dog is walking on a couch',
 'a man is pouring a can of food',
 'a woman is cutting a potato',
 'a man is playing a guitar',
 'a woman is cutting a potato',
 'a man is riding a horse',
 'a woman is cutting a potato',
 'a man is riding a horse',
 'a man is cutting a potato',
 'a man is playing a guitar',
 'a man is riding a horse',
 'a man is playing a guitar',
 'a man is pouring sauce into a pot',
 'a man is riding a horse',
 'a man is riding a horse',
 'a baby is eating food',
 'a man is riding a horse',
 'a man is cutting a potato',
 'a man is playing a flute',
 'a man is riding a horse',
 'a man is playing a guitar',
 'a cat is playing with a dog',
 'a woman is cutting a potato',
 'a man is riding a horse',
 'a man is riding a horse',
 'a man is cutting a potato',
 'a cat is playing with a dog',
 'a baby is playing with a baby',
 'a woman is pouring oil into a pot',
 'a man is pouring sauce into a bowl',
 'a man is riding a horse',
 'a man is playin

In [41]:
np.load('./clip_index/testIndex.npy')[:10]

array([['1300', '8MVo7fje_oE_153_158'],
       ['1301', 'qPXynwa_2iM_15_25'],
       ['1302', 'eVSQiPbepXg_83_93'],
       ['1303', 'J_evFB7RIKA_104_120'],
       ['1304', 'tYh4iDFgmEE_50_54'],
       ['1305', 'zYcY4mjLpxU_45_55'],
       ['1306', '3mbBbFH9fAg_21_29'],
       ['1307', 'qeKX-N1nKiM_123_130'],
       ['1308', '8PQiaurIiDM_247_255'],
       ['1309', 'fX5G_JwPlLo_640_660']], dtype='|S21')